# Learn the code - Part 3

---
**Purpose:** Learn how to measure water level with gnssrefl using GNSS data 

In [1]:
import os

from gnssrefl.utils import check_environment, set_environment, get_sys
from pathlib import Path

# Making sure environment variables are set - this is required to run the gnssrefl code
notebook_dir = Path.cwd().parents[0]

exists = check_environment()
if exists == False:
    #If you are running this locally - make sure the items in the exe folder have execution permissions
    set_environment(refl_code=str(notebook_dir),
                    orbits=str(notebook_dir / "orbits"),
                    exe=str(notebook_dir / "exe"))

# Set local variable of refl_code location
refl_code_loc = os.environ['REFL_CODE']
print("refl_code location:", refl_code_loc)

# import gnssrefl functions
from gnssrefl.rinex2snr_cl import rinex2snr
from gnssrefl.quickLook_cl import quicklook
from gnssrefl.gnssir_cl import gnssir
from gnssrefl.daily_avg_cl import daily_avg
from gnssrefl.installexe_cl import installexe

#@formatter:off
%matplotlib inline

environment variable ORBITS set to path /Users/enloe/UNAVCO_Projects/git/gnssrefl_jupyter/orbits 
environment variable REFL_CODE set to path /Users/enloe/UNAVCO_Projects/git/gnssrefl_jupyter 
environment variable EXE set to path /Users/enloe/UNAVCO_Projects/git/gnssrefl_jupyter/bin/exe


In [2]:
# Download the executables
try:
    os.environ['DOCKER']
except KeyError:
    sys = get_sys()
    installexe(sys)

Your executable environment area:  /Users/enloe/UNAVCO_Projects/git/gnssrefl_jupyter/bin/exe
You already have this executable: CRX2RNX
Executable already exists: teqc
You already have this executable: gfzrnx


**Station:**
We will be using station **ross**. It is operated by [NRCAN](https://www.nrcan.gc.ca). 
This [map](https://webapp.geod.nrcan.gc.ca/geod/data-donnees/cacs-scca.php?locale=en)
gives you an overview of GNSS stations operated by NRCAN. Use the plus sign on the map 
to look more closely at Lake Superior. Find **ross** and click on it (station M023004). 
If you scroll down, you will see a photo of the monument. 

NRCAN is operating what I would call a "legacy" GNSS instrument. This means it only tracks the original GPS signals that were designed in the 1970s. This means none of the enhanced GPS signals (L2C and L5) available since 2005 are provided. Furthermore, there are no signals from Glonass, Galileo, or Beidou. The bottom line is that you will be using only the L1 GPS signal, which leaves you with ~15% of what would be available from a modern multi-GNSS unit. The sample rate - 30 seconds - limits what kind of reflectometry you can do. For the purposes of this homework, it restricts the RH to values less than ~10 meters.

**Azimuth/Elevation Mask**

Next, let's get an idea of what this site looks like from a reflections viewpoint. 

Run the cell below (starts with %%html) to view the gnss-reflections webapp.
Use the geoid tab in the webapp to
get an idea of its surroundings. You can enter the station coordinates by hand if you know them, 
but since **ross** is part of a public archive known to geodesists, coordinates have been stored in the webapp.
Just type in **ross** for the station name. Make a note of the station 
latitude, longitude, and ellipsoidal height that is returned by the 
webapp because you will need it later. Although the elevation above sea level of 
the site is ~186 meters, from the photo you know already this is not the value 
we will want to use for our reflections study. We will start with our common 
sense, look at the data, and iterate if necessary.

In [ ]:
# save the variable for the station name since it will be used for the rest of the notebook
station = 'ross'

# fill in the values for these variables from the output of the web app below
lat = 48.83372945
long = -87.5195988
height = 149.835

In [1]:
%%html
<iframe src="https://gnss-reflections.org/geoid" width="1000" height="600"></iframe>

Use the [reflection zone section of the web app](https://gnss-reflections.org/rzones) in the cell below to get an idea
of what reflection zones are possible for this site. We cannot use the default sea level reflection 
value, so you need to set a Reflector Height (RH) value. Based on the photograph, try values that
you think are reasonable. You don't want your reflection zones to cross 
a dock or the nearby boats, so you should also rerun it with different azimuth limits. Don't worry about it too much as we will get feedback from the actual GPS data.

Make a note of:

<UL>
<LI>RH
<LI>elevation angle values that give water coverage without interference from docks/boats
<LI>azimuth angle values that cover open water without interference
<LI>the DECIMAL latitude, longitude, and height (from the geoid webapp).
<LI>we can only use L1 GPS data at this site 
<LI>We can't estimate RH larger than 10 meters because of the sampling rate
</UL>

In [ ]:
%%html
<iframe src="https://gnss-reflections.org/rzones" width="800" height="800"></iframe>

**Using gnssrefl**

Now let's look at the **ross** data. We need to pick up a RINEX file and strip out the 
SNR data.  We use the <code>rinex2snr</code> for this purpose.  Use -h if you want to 
see the options for this module. We will throw caution to the winds and see if the defaults will work. 
The only required inputs are the station name (ross), the year (2020) and day of year (150) 
(note: to convert from year and day of year to year, 
month, day and vice versa, try the modules <code>ydoy</code> and 
<code>ymd</code>). 

In this case the RINEX data are available from both sopac and nrcan. Try the <code>archive</code> option.

In [ ]:
rinex2snr?

In [ ]:
year = 2020
doy = 150
# if you would like to pick an archive then you can 
# archive = 'nrcan'
# archive = 'sopac'

# if you choose to use an archive then you must add the parameter archive example:
# args = run_gnssrefl.rinex2snr(station, year, doy, archive=archive)

# otherwise, the only required parameters are station, year, and doy
rinex2snr(station, year, doy)

---
Once you have successfully created a SNR file, run <code>quickLook</code>.

The quicklook plots consist of two graphical representations of the data. The first is 
periodograms for the four geographic quadrants (northwest, northeast, and so on). 
You are looking for nice clean (and colorful) peaks. Color means they have 
passed Quality Control (QC). Gray lines are satellite tracks that failed QC. The second plot summarizes the 
RH retrievals and how the QC metrics look compared to 
the defaults. In this case the x-axis is azimuth in degrees.

In [ ]:
# Plotting using plt=True
values, metrics = quicklook(station, year, doy=doy, plt=True)

From these plots, how does the correct *RH* value compare with the one you assumed earlier when you 
were trying out the webapp?  How about the azimuths?  Go back to the reflection zone webapp and 
make sure you are happy with your azimuth and elevation angle selections.

Next we need to save our <code>gnssrefl</code> analysis strategy using 
<code>make_json_input</code>. At a minimum you need to know 
the latitude, longitude, and height for the station (we saved these as variables earlier so they can easily be accessed at this point). However, your analysis strategy can and should
be improved by setting some parameters.

*Hints:*

* Check the documentation to see how to set the elevation angles and RH limits. You can run the following cell to see the parameters.

* Since we can only use L1 data, you should use the <code>l1=True</code>.

* You will need to hand edit the azimuths in the json file. You want to cut up your azimuth range in 60-90 degree chunks.  So if you wanted to use the region  for 90-270 degrees, you should say 90-180 and 180-270. You can use smaller chunks, but I generally do not use less than 45 degree azimuth chunks.

In [ ]:
# run this cell to view the available parameters and current defaults
make_json?

In [ ]:
make_json(station, lat, long, height, l1=True)

In [ ]:
# This is the json file that was created with the defaults + parameters you set above
json_file = f'{refl_code_loc}/input/{station}.json'
with open(json_file, "r") as myfile:
    file = json.load(myfile)

file

In [ ]:
# Now lets edit the json file
with open(json_file, "r") as myfile:
    file = json.load(myfile)

# Here is where we can 'hand edit' values in the json file
# lets edit the azimuths
file['azval'] = [90, 180, 180, 270]

with open(json_file, 'w') as f:
    json.dump(file, f, indent=4)

# now lets view it again and note the difference
with open(json_file, "r") as myfile:
    file = json.load(myfile)

file

Now run <code>gnssir</code> for the year 2020/doy 150. 
This module is meant for *routine analysis* and thus there are not a lot
of bells and whistles. However, it is good practice to see that something is actually 
created (the screen output will tell you where it is).

In [ ]:
year = 2020
doy = 150
gnssir(station, year, doy, plt=True, screenstats=False)

**Extra Credit:**

The <code>gnssir</code> output tells you the vertical distance between the GPS antenna and 
the lake for each successful satellite track. That is not 
super exciting; it is a little more interesting to see if it changes over time, which means 
you need to analyze a bit more data. 

* use <code>rinex2snr</code> to make SNR files for the same year, but now do doy 120 through 290. You can use the <code>doy_end</code> parameter to do that. And use <code>weekly=True</code> to make fewer files (which will make everything much faster). Why did I pick those dates? Mostly to avoid snow (yeap, it snows up there!) 

In [ ]:
doy = 120
doy_end = 290

rinex2snr(station, year, doy=doy, doy_end=doy_end, weekly=True)

* run <code>gnssir</code> for those dates. You do not need the weekly option here - you can just specify 120 through 290. It will look for every day, but if it doesn't find it, it just looks for the next day, etc.

In [ ]:
# station, year, doy, and doy_end variables are already set.
gnssir(station, year, doy, doy_end=doy_end, screenstats=False)

* You can now use the <code>daily_avg</code> to make a daily average for the lake level on each day you analyzed.  

In [ ]:
daily_avg?

In [ ]:
daily_avg(station, medfilter=.25, ReqTracks=20, txtfile=f'{station}-dailyavg.txt')

# set a variable to where the file is saved
dailyavg_filepath = f'{refl_code_loc}/Files/{station}-dailyavg.txt'

* Let's plot the daily average file on each day analyzed. You can use the function read_rh_files from the gnssrefl_helpers module to read in the file.

In [ ]:
data = gnssrefl_helpers.read_rh_files(dailyavg_filepath)
# set data to pandas dataframe
df = pd.DataFrame(data, index=None, columns=['dates', 'rh'])
df.head()

In [ ]:
# Now plot
plt.figure(figsize=(8, 8))
g = sns.scatterplot(x='dates', y='rh', data=df, legend=False)
g.set_title('Ross Daily Mean Reflector Height')
g.set_ylabel('Reflector Height (m)')
g.set_ylim(4.55, 4.26)  # flip y axis so so 'up' is water levels rising
plt.show()

**Extra Extra Credit:**

Compare your results with the [lake gauge data.](https://www.isdm-gdsi.gc.ca/isdm-gdsi/twl-mne/inventory-inventaire/sd-ds-eng.asp?no=10220&user=isdm-gdsi&region=CA)